## Постановка задачи

1. Предобработать данные из датасета **Credit card** и разделить их на тренировочный и тестовый датасеты.
2. Создать простейший классификатор без регуляризации и обучить его на тренировочных данных.
3. Создать простейший классификатор с регуляризацией и обучить его на тех же тренировочных данных.
4. Сравнить точность двух классификаторов.
5. Оценить **precision** и **recall** для каждого из результатов.

In [321]:
import numpy as np # импортируем библиотеку Numpy
import pandas as pd # импортируем библиотеку Pandas
import matplotlib.pyplot as plt # импортируем библиотеку Matplotlib

In [322]:
credit_card_data = pd.read_csv('https://raw.githubusercontent.com/Andruchelli/Credit_card/main/UCI_Credit_Card.csv') # прочитаем файл с данными

In [323]:
credit_card_data.head() # проверяем, что файл прочитался

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [324]:
credit_card_data.shape # смотрим размерность датасета с тренировочными данными

(30000, 25)

Считаем количество ячеек в столбцах датасета со значением NaN, которое не востребовано для дальнейшей предобработки данных

In [325]:
np.sum(credit_card_data.isna())

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default.payment.next.month    0
dtype: int64

Ячейки со значением NaN отсутствуют, поэтому мы можем продолжить работу со всеми данными датасета.

При проверке размерности нашего датасета мы узнали, что в нём содержится 30 тысяч строк. Считаем, что это слишком большой объём данных и, чтобы не перегружать систему, возьмём только часть данных. Например, 300 строк

In [326]:
credit_card_data = credit_card_data.iloc[:300] # берём первые 300 строк

In [327]:
credit_card_data.shape # проверяем размерность нового датасета

(300, 25)

Объединим несколько столбцов, чтобы сократить их количество. Например, столбцы **PAY_AMT**, содержащие данные о сумме предыдущего платежа. Сложим их, чтобы получить общую сумму платежей владельца кредитной карты за весь период

In [328]:
credit_card_data['SUM_PAY'] = credit_card_data['PAY_AMT1'] + credit_card_data['PAY_AMT2'] + credit_card_data['PAY_AMT3'] + credit_card_data['PAY_AMT4'] + credit_card_data['PAY_AMT5'] + credit_card_data['PAY_AMT6'] # в датафрейме создаём новую колонку (сумма)
credit_card_data = credit_card_data.drop(['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'], axis=1) # после получения новой фичи (features) нужно сбросить старые колонки; используем функцию drop, axis=1 сбрасывает колонки

In [329]:
credit_card_data.head() # убедимся, что новая колонка добавилась, а старые удалились

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,default.payment.next.month,SUM_PAY
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,1,689.0
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,1,5000.0
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,0,11018.0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,0,8388.0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,0,59049.0


Для проведения дальнейших исследований разделим полученный датасет на тренировочные и тестировочные данные.

Возьмём данные по столбцам **LIMIT_BAL** (сумма предоставленного кредита) и **SUM_PAY** (сумма всех платежей) и попробуем определить пол плательщика (**SEX**): если значение равно "1", то это мужчина, если значение равно "2", то женщина

In [330]:
# 2/3 данных испольуем в качестве тренировочных данных
X_train1 = credit_card_data[['LIMIT_BAL', 'SUM_PAY']][:200].values # разбиваем датасет на тренировочные данные X (features)
Y_train1 = credit_card_data['SEX'][:200] # и тренировочные результаты Y (labels)
# 1/3 данных отдаём в тестировочные данные
X_test1 = credit_card_data[['LIMIT_BAL', 'SUM_PAY']][200:].values
Y_test1 = credit_card_data['SEX'][200:]

print("X_train1.shape: {}".format(X_train1.shape))
print("Y_train1.shape: {}".format(Y_train1.shape))
print("X_test1.shape: {}".format(X_test1.shape))
print("Y_test1.shape: {}".format(Y_test1.shape))

X_train1.shape: (200, 2)
Y_train1.shape: (200,)
X_test1.shape: (100, 2)
Y_test1.shape: (100,)


С помощью классификатора логистической регресии предсказываем данные (без регуляризации)

In [331]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression()

clf1.fit(X_train1, Y_train1) # обучаем модель

LogisticRegression()

In [332]:
Y_pred1 = clf1.predict(X_test1) # делаем предсказание

In [333]:
Y_pred1

array([2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2,
       2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2])

Для оценки точности регрессионной модели используем метод **accuracy_score** из библиотеки **sklearn**

In [345]:
from sklearn.metrics import accuracy_score

train_accuracy1 = accuracy_score(Y_train1, clf1.predict(X_train1))
print(f'train accuracy:', train_accuracy1)

test_accuracy1 = accuracy_score(Y_test1, clf1.predict(X_test1))
print(f'test accuracy:', test_accuracy1)

#print("train accuracy: {}".format(accuracy_score(Y_train1, clf1.predict(X_train1)))) # Y_train (реальные выходные данные labels) и clf.predict(X_train) (предсказанные данные) 
#print("test accuracy: {}".format(accuracy_score(Y_test1, clf1.predict(X_test1))))

train accuracy: 0.54
test accuracy: 0.66


In [346]:
from sklearn.metrics import precision_score

train_precision1 = precision_score(Y_train1, clf1.predict(X_train1))
print(f'train precision:', train_accuracy1)

test_precision1 = precision_score(Y_test1, clf1.predict(X_test1))
print(f'test precision:', test_precision1)

#print("train precision: {}".format(precision_score(Y_train1, clf1.predict(X_train1))))
#print("test precision: {}".format(precision_score(Y_test1, clf1.predict(X_test1))))

train precision: 0.54
test precision: 0.56


In [347]:
from sklearn.metrics import recall_score

train_recall1 = recall_score(Y_train1, clf1.predict(X_train1))
print(f'train_recall:', train_recall1)

test_recall1 = recall_score(Y_test1, clf1.predict(X_test1))
print(f'test_recall:', test_recall1)

#print("train recall: {}".format(recall_score(Y_train1, clf1.predict(X_train1))))
#print("test recall: {}".format(recall_score(Y_test1, clf1.predict(X_test1))))

train_recall: 0.2916666666666667
test_recall: 0.3783783783783784


Из результатов вычислений видно, что с тренировочными данными модель справилась с точностью 0.54, а с тестовыми - 0.66.

Значение точности (**Precision**) для тренировочных данных составило 0.54, а для тестовых данных - 0.56.

Значение охвата (**Recall**) для тренировочных данных данной модели составило 0.29, а для тестовых - 0.38.

Теперь попробуем применить к нашей модели классификатор с регуляризацией. Для этой цели выберем SDG-классификатор, основанный на методе стохастического градиентного спуска

In [348]:
# 2/3 данных испольуем в качестве тренировочных данных
X_train2 = credit_card_data[['LIMIT_BAL', 'SUM_PAY']][:200].values # разбиваем датасет на тренировочные данные X (features)
Y_train2 = credit_card_data['SEX'][:200] # и тренировочные результаты Y (labels)
# 1/3 данных отдаём в тестировочные данные
X_test2 = credit_card_data[['LIMIT_BAL', 'SUM_PAY']][200:].values
Y_test2 = credit_card_data['SEX'][200:]

print("X_train2.shape: {}".format(X_train2.shape))
print("Y_train2.shape: {}".format(Y_train2.shape))
print("X_test2.shape: {}".format(X_test2.shape))
print("Y_test2.shape: {}".format(Y_test2.shape))

X_train2.shape: (200, 2)
Y_train2.shape: (200,)
X_test2.shape: (100, 2)
Y_test2.shape: (100,)


In [349]:
from sklearn.linear_model import SGDClassifier # импортируем классификатор

In [350]:
clf2 = SGDClassifier(verbose=4, max_iter=500) # устанавливаем значение в 500 итераций

In [351]:
clf2.fit(X_train2, Y_train2) # обучаем модель на тренировочных данных

-- Epoch 1
Norm: 1133252.18, NNZs: 2, Bias: 45.704999, T: 200, Avg. loss: 87673362282.778824
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 2802023.94, NNZs: 2, Bias: -9.705950, T: 400, Avg. loss: 72058776266.164932
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 602826.46, NNZs: 2, Bias: 17.827723, T: 600, Avg. loss: 79101989778.295425
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 1746639.53, NNZs: 2, Bias: 58.009418, T: 800, Avg. loss: 67513853756.163635
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 2166075.40, NNZs: 2, Bias: 70.142841, T: 1000, Avg. loss: 46567955765.488846
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 900156.96, NNZs: 2, Bias: 99.546312, T: 1200, Avg. loss: 53428341302.049240
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 1801577.17, NNZs: 2, Bias: 117.170995, T: 1400, Avg. loss: 57444277895.837219
Total training time: 0.00 seconds.
-- Epoch 8
Norm: 1382308.05, NNZs: 2, Bias: 101.871014, T: 1600, Avg. loss: 59162939780.255676
Total t

SGDClassifier(max_iter=500, verbose=4)

In [352]:
from sklearn.metrics import accuracy_score # считаем тончость с помощью метода accuracy_score

train_accuracy2 = accuracy_score(Y_train2, clf2.predict(X_train2))
print(f'train accuracy:', train_accuracy2)

test_accuracy2 = accuracy_score(Y_test2, clf2.predict(X_test2))
print(f'test accuracy:', test_accuracy2)

#print("train accuracy: {}".format(accuracy_score(Y_train2, clf2.predict(X_train2)))) # Y_train (реальные выходные данные labels) и clf.predict(X_train) (предсказанные данные) 
#print("test accuracy: {}".format(accuracy_score(Y_test2, clf2.predict(X_test2))))

train accuracy: 0.58
test accuracy: 0.57


Из результатов вычислений видно, что с тренировочными данными классификатор с регуляризацией справился с точностью 0.58, а с тестовыми - 0.57

In [355]:
from sklearn.metrics import precision_score

train_precision2 = precision_score(Y_train2, clf2.predict(X_train2))
print(f'train precision:', train_precision2)

test_precision2 = precision_score(Y_test2, clf2.predict(X_test2))
print(f'test precision:', test_precision2)

#print("train precision: {}".format(precision_score(Y_train2, clf2.predict(X_train2))))
#print("test precision: {}".format(precision_score(Y_test2, clf2.predict(X_test2))))

train precision: 0.5555555555555556
test precision: 0.44642857142857145


Из полученных данных видно, что у модели с тренировочными данными точность (**Precision**) равна 0.56, а с тестовыми - 0.45

In [354]:
from sklearn.metrics import recall_score

train_recall2 = recall_score(Y_train2, clf2.predict(X_train2))
print(f'train recall:', train_recall2)

test_recall2 = recall_score(Y_test2, clf2.predict(X_test2))
print(f'test recall:', test_recall2)

#print("train recall: {}".format(recall_score(Y_train2, clf2.predict(X_train2))))
#print("test recall: {}".format(recall_score(Y_test2, clf2.predict(X_test2))))

train recall: 0.625
test recall: 0.6756756756756757


Охват (**Recall**) в данной модели с тренировочными данными получился равен единиице 0.63, а с тестовыми - 0.68.

Сравним полученные результаты:

In [360]:
df = pd.DataFrame({'Model': ['Первая', 'Вторая'], 'train accuracy': [train_accuracy1, train_accuracy2], 'test accuracy': [test_accuracy1, test_accuracy2],
                  'train precision': [train_precision1, train_precision2], 'test precision': [test_precision1, test_precision2],
                  'train recall': [train_recall1, train_recall2], 'test recall': [test_recall1, test_recall2]})
df

,Model,train accuracy,test accuracy,train precision,test precision,train recall,test recall
0,Первая,0.54,0.66,0.538462,0.560000,0.291667,0.378378
1,Вторая,0.58,0.57,0.555556,0.446429,0.625000,0.675676


# Выводы:

1. Значение точности (**accuracy**) для тренировочных данных получилось больше во **второй** модели (с регуляризацией), но для тестировочных данных ближе к "1" находится значение **первой** модели (без регуляризации).
2. Значение точности (**precision**) для тренировочных данных оказалось почти одинаковым для обеих моделей, а для тестировочных данных лучше результат оказался у **первой** модели (без регуляризации).
3. Самый большой разброс в данных получился в результатах охвата (**recall**), в которых **вторая** модель (с регуляризацией) в обоих случаях (и в тренировочных, и в тестировочных данных) дала более высокий результат.